In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('./propagation.csv', header=0,  index_col=0)
final = pd.DataFrame(data)

final.tail(10)

,experiment,start,end,topology,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,bandwidth,messageOverhead,propagationTime
202,653,2023-11-21 21:34:06.4672540,2023-11-21 22:04:06.4816988,general,16,8,15,16,8,4,0.75,0.5,166.092000,329302,1812.411298
203,654,2023-11-21 22:07:40.2346165,2023-11-21 22:37:40.2385545,general,5,4,7,3,2,2,0.25,1.0,161.342425,289852,1949.999921
204,655,2023-11-21 22:41:14.2028913,2023-11-21 23:11:14.2104657,general,5,4,7,3,2,2,0.25,1.0,107.576214,355050,1480.055445
205,656,2023-11-21 23:14:48.2294649,2023-11-21 23:44:48.2363093,general,5,4,7,3,2,2,0.25,1.0,218.079171,690868,1898.799444
206,657,2023-11-21 23:48:22.6068989,2023-11-22 00:18:22.6099262,general,6,4,8,7,5,4,0.25,3.0,343.137756,351292,1824.503531
207,658,2023-11-22 00:21:57.0236579,2023-11-22 00:51:57.0268426,general,6,4,8,7,5,4,0.25,3.0,41.519371,29993,1539.963089
208,661,2023-11-22 02:02:40.2343885,2023-11-22 02:32:40.2368451,general,6,3,8,2,2,4,0.25,0.5,4.343883,48864,1069.693480
209,662,2023-11-22 02:36:14.2823737,2023-11-22 03:06:14.2932442,general,6,3,8,2,2,4,0.25,0.5,37.513662,678236,1547.234062
210,663,2023-11-22 03:09:48.5789046,2023-11-22 03:39:48.5860069,general,21,16,24,5,16,8,0.50,1.0,45.392599,897920,1308.499583
211,664,2023-11-22 03:43:22.2716142,2023-11-22 04:13:22.2769946,general,21,16,24,5,16,8,0.50,1.0,6.162759,123227,1153.951237


In [4]:
data = pd.read_csv('../../datasets/timeSeries1s.csv', header=0,  index_col=0)
message = pd.DataFrame(data)

# idx = [2.0]
# message = message[message["type"].isin(idx)]
# message = message.drop(columns=["type", "identifier"])

message.head(10)

,identifier,experiment,type,_time,count
0,0,618,11.0,2023-11-16 21:19:03+00:00,76
1,0,618,11.0,2023-11-16 21:19:04+00:00,1390
2,0,618,11.0,2023-11-16 21:19:07+00:00,14
3,0,618,12.0,2023-11-16 21:19:04+00:00,7
4,0,618,12.0,2023-11-16 21:19:06+00:00,82
5,0,618,12.0,2023-11-16 21:19:07+00:00,163
6,0,618,16.0,2023-11-16 21:14:51+00:00,1
7,0,618,16.0,2023-11-16 21:19:10+00:00,1
8,0,618,16.0,2023-11-16 21:20:28+00:00,2
9,0,618,16.0,2023-11-16 21:20:46+00:00,1


In [5]:
events = message.groupby(['experiment', 'type']).agg({'count':['sum']})
events.columns = events.columns.droplevel(0)
events.reset_index(inplace=True)

events.head(10)

,experiment,type,sum
0,385,2.0,338415
1,385,3.0,46100
2,385,11.0,1437
3,385,12.0,180
4,385,16.0,12
5,385,17.0,56795
6,386,2.0,959077
7,386,3.0,127709
8,386,11.0,1468
9,386,12.0,239


In [6]:
pivoted = pd.pivot_table(events,index=["experiment"], columns="type", values="sum", margins=False).fillna(0)
pivoted = pivoted.reset_index().rename_axis(None, axis=1)

pivoted.head(100)

,experiment,2.0,3.0,11.0,12.0,16.0,17.0
0,385,338415.0,46100.0,1437.0,180.0,12.0,56795.0
1,386,959077.0,127709.0,1468.0,239.0,34.0,161937.0
2,389,106497.0,18224.0,0.0,0.0,9.0,37316.0
3,390,2473589.0,342351.0,1497.0,222.0,26.0,172331.0
4,391,1681977.0,242460.0,1537.0,125.0,17.0,109756.0
...,...,...,...,...,...,...,...
95,510,1656353.0,135478.0,2123.0,0.0,2.0,10329.0
96,511,237418.0,19911.0,2126.0,0.0,1.0,1057.0
97,512,2572194.0,199414.0,2052.0,0.0,1.0,651.0
98,513,2432321.0,194907.0,2111.0,0.0,6.0,6102.0


In [8]:
eventsTS = pivoted.rename(columns={2.0:'messageReceived', 3.0: 'messageDuplicated', 11.0: 'graft', 12.0: 'prune', 16.0: 'iwant', 17.0:'ihave'})
eventsTS.to_csv('events.csv')
eventsTS.head(500)

,experiment,messageReceived,messageDuplicated,graft,prune,iwant,ihave
0,385,338415.0,46100.0,1437.0,180.0,12.0,56795.0
1,386,959077.0,127709.0,1468.0,239.0,34.0,161937.0
2,389,106497.0,18224.0,0.0,0.0,9.0,37316.0
3,390,2473589.0,342351.0,1497.0,222.0,26.0,172331.0
4,391,1681977.0,242460.0,1537.0,125.0,17.0,109756.0
...,...,...,...,...,...,...,...
213,658,29993.0,6086.0,0.0,0.0,0.0,68649.0
214,661,48864.0,7813.0,548.0,0.0,0.0,6.0
215,662,678236.0,67489.0,541.0,0.0,0.0,36.0
216,663,897920.0,81701.0,539.0,0.0,3.0,6.0


In [9]:
finalOve = final.merge(eventsTS, on='experiment')
# finalOve = finalOve.rename(columns={'sum' : 'messageOverhead'})#.drop(columns=['std'])

finalOve.to_csv('consolidated.csv')
finalOve.head(10)

,experiment,start,end,topology,d,dlo,dhi,dscore,dlazy,dout,...,interval,bandwidth,messageOverhead,propagationTime,messageReceived,messageDuplicated,graft,prune,iwant,ihave
0,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518,unl,8,6,12,4,8,2,...,1.0,285.933511,2473589,2150.636202,2473589.0,342351.0,1497.0,222.0,26.0,172331.0
1,391,2023-10-04 19:25:37.1533399,2023-10-04 19:55:37.1555856,unl,8,6,12,4,8,2,...,1.0,195.675311,1681977,1886.452761,1681977.0,242460.0,1537.0,125.0,17.0,109756.0
2,392,2023-10-04 19:59:12.6589825,2023-10-04 20:29:12.6608307,unl,8,6,12,4,8,2,...,1.0,86.268800,1379082,1796.105503,1379082.0,130115.0,2129.0,0.0,2.0,25169.0
3,393,2023-10-04 20:32:47.3098616,2023-10-04 21:02:47.3149178,unl,8,6,12,4,8,2,...,3.0,108.486362,1948016,2027.536181,1948016.0,192998.0,2138.0,0.0,0.0,2278.0
4,394,2023-10-04 21:06:22.7755634,2023-10-04 21:36:22.7788800,unl,8,6,12,4,8,2,...,3.0,94.085382,1843504,2079.683079,1843504.0,164486.0,2137.0,0.0,1.0,4868.0
5,395,2023-10-04 21:39:58.6528165,2023-10-04 22:09:58.6545536,unl,8,6,12,4,8,2,...,3.0,36.928853,687571,1934.724939,687571.0,64290.0,2132.0,0.0,0.0,2182.0
6,396,2023-10-04 22:13:33.8146704,2023-10-04 22:43:33.8174094,unl,16,8,20,16,8,4,...,1.0,79.777656,1428674,2048.156118,1428674.0,140085.0,2141.0,0.0,1.0,3515.0
7,397,2023-10-04 22:47:09.3837110,2023-10-04 23:17:09.3868442,unl,16,8,20,16,8,4,...,1.0,69.534879,1428254,1808.903694,1428254.0,124371.0,2138.0,0.0,0.0,792.0
8,398,2023-10-04 23:20:44.6001719,2023-10-04 23:50:44.6033804,unl,16,8,20,16,8,4,...,1.0,7.107765,141417,384.884444,141417.0,12794.0,0.0,0.0,0.0,0.0
9,402,2023-10-05 01:35:05.2471450,2023-10-05 02:05:05.2498822,unl,16,8,20,16,8,4,...,0.5,75.661552,289177,2075.841833,289177.0,90791.0,1227.0,42.0,11.0,45400.0
